In [1]:
import numpy as np
import os, sys
import pickle, functools, operator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random #shuffle
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from netCDF4 import Dataset

import numpy as np
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from netCDF4 import Dataset

In [2]:
def CMIPdata(Xdata, Ydata, out , myform):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')
    sst_1 = np.zeros((myform,140,12,24,72))

    #首年序列
    #i= 21个模式
    for i in range(myform):
        sst_1[i,:,:,:,:] = inp1.variables['sst1'][1+141*i:141+141*i,0:12,:,:]
    #(21,140,12,24,72)
    #首年序列

    
    #flatted扁平化                                             
    sst_2 = np.zeros((myform,1680,24,72))
    for i in range(myform):
        for j in range(140):
            sst_2[i,j*12:(j+1)*12,:,:] = sst_1[i,j,:,:,:]
    
        #(21,1680,24,72)
        #每个模式 139*12+1 = 1680 个序列 序列长度为12

    #丢弃1个月 1862.1
    sst_2 = sst_2[:,1:,:,:]
    #(form,1679,24,72)


    winsize = 12
    #winnum = 1679-winsize +1 #1668
    """跳跃六个月滑窗 = 6"""
    ts=6 
    #(1679-12)/6+1 =27
    winnum=int((1679-12)/ts +1) 
    
    sst_3 = np.zeros((myform,winnum,12,24,72))
    for i in range(myform):
        for j in range(winnum):
            #print(j)
            sst_3[i,j,:,:,:] = sst_2[i,(j*ts):(j*ts)+winsize,:,:]
            

    sst_4 = np.zeros((int(winnum*myform),12,24,72))
    #t300_4 = np.zeros((int(winnum*myform),12,24,72))
    for i in range(myform):
        sst_4[i*winnum:(i+1)*winnum,:,:,:] = sst_3[i,:,:,:,:]


    #sst_4 = sst_4.swapaxes(1, 3)
    trX = sst_4
    del sst_1,sst_2,sst_3,sst_4
    #trX = trX.reshape(int(winnum*myform),12,1728)

    #保存np数组
    #(5859, 12, 24, 72, 2)
    #np.save("./CMIPdata/CMIP_trX_21_ts6_out.npy",trX) 

    #label
    #平铺
    pr_1 = np.zeros(((myform),1692))#21,1692
    for i in range(myform):
        for j in range(141):
            pr_1[i,j*12:(j+1)*12] = inp2.variables['pr'][(141*i)+j,:,0,0]
            
    #out
    pr_2 =  np.zeros((myform,winnum,out))#form,1668,1
    for i in range(myform):
        for j in range(winnum):
            pr_2[i,j,:] = pr_1[i,(j*ts):(j*ts)+out]

    trY = np.zeros((int(winnum*myform),out))#form*1668,1
    for i in range(myform):
        trY[i*winnum:(i+1)*winnum,:] = pr_2[i,:,:]

    #np.save("./CMIPdata/CMIP_trY_21_ts6_out%s.npy"%out,trY)
    trY_decoder_input = np.zeros((int(winnum*myform),out))
    trY_decoder_input[:,1:] = trY[:,:-1]

    #return trX[1:,:,:] ,trY.reshape(-1,24,1)[1:,:,:] ,trY_decoder_input
    return trX ,trY.reshape(-1,24,1) ,trY_decoder_input.reshape(-1,24,1)

In [3]:
def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    #t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    #t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    #t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        #t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    #t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    #t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        #t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    #sst_33 = sst_33.swapaxes(1, 3)

    testX = sst_33#384
    #testX[:,:,:,1] = t300_33.reshape(384,12,1728)
    del sst_11,sst_22,sst_33
    #del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY


In [4]:
trX, trY, trY_decoder_input = CMIPdata('/home/d/Q/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

In [5]:
testX, testY = GOSDAdata('/home/d/Q/saconvlstm/GODAS.input.36mon.1980_2015.nc',\
                         '/home/d/Q/saconvlstm/GODAS.label.12mon.1982_2017.nc', 24)

In [6]:
print(testX.shape)
print(testY.shape)

(384, 12, 24, 72)
(384, 24, 1)


In [7]:
print(trX.shape)
print(trY.shape)
print(trY_decoder_input.shape)

(5838, 12, 24, 72)
(5838, 24, 1)
(5838, 24, 1)


In [7]:
trX = np.around(trX, decimals=3)
trY = np.around(trY, decimals=3)
trY_decoder_input = np.around(trY_decoder_input, decimals=3)

testX = np.around(testX, decimals=3)
testY = np.around(testY, decimals=3)

# model

In [8]:
input_cnn = Input(shape=(24, 72,1))

x = layers.Conv2D(30, (8,4), activation='tanh', padding='SAME')(input_cnn)
x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = layers.Conv2D(30, (4,2), activation='tanh', padding='SAME')(x)
x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = layers.Conv2D(30, (4,2), activation='tanh', padding='SAME')(x)
x = tf.reshape(x, [-1, 30 * 18 * 6])
outputs_cnn = x
model_cnn = Model(input_cnn,outputs_cnn)

2021-11-05 02:18:41.998814: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-05 02:18:42.018422: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 3799900000 Hz
2021-11-05 02:18:42.019135: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56257a3b4180 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-05 02:18:42.019161: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [26]:
x1 = layers.Conv2D(50, (8,4), activation='tanh', padding='SAME')(input_cnn)
x1 = tf.nn.max_pool(x1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x1 = layers.Conv2D(50, (4,2), activation='tanh', padding='SAME')(x1)
x1 = tf.nn.max_pool(x1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x1 = layers.Conv2D(50, (4,2), activation='tanh', padding='SAME')(x1)
x1 = tf.reshape(x1, [-1, 50 * 18 * 6])
outputs_cnn1 = x1
model_cnn1 = Model(input_cnn,outputs_cnn1)
#model_cnn1.summary()


In [9]:
_input = Input(shape=(12, 24, 72))
_input1 = tf.reshape(_input, [-1, 24, 72])
x = model_cnn(_input1)
#x1 = model_cnn1(_input1)
output = tf.reshape(x, [-1,12, 30 * 18 * 6])
#output1 = tf.reshape(x1, [-1,12, 50 * 18 * 6])
#output = tf.concat([output,output1],2)#(None, 12, 6480)

In [9]:
_input = Input(shape=(72,24,12))
x1 = model_cnn(_input[:,:,:,0])
x2 = model_cnn(_input[:,:,:,1])
x3 = model_cnn(_input[:,:,:,2])
x4 = model_cnn(_input[:,:,:,3])
x5 = model_cnn(_input[:,:,:,4])
x6 = model_cnn(_input[:,:,:,5])
x7 = model_cnn(_input[:,:,:,6])
x8 = model_cnn(_input[:,:,:,7])
x9 = model_cnn(_input[:,:,:,8])
x10 = model_cnn(_input[:,:,:,9])
x11 = model_cnn(_input[:,:,:,10])
x12 = model_cnn(_input[:,:,:,11])

x1 = tf.reshape(x1, [-1, 1, 3240])
x2 = tf.reshape(x2, [-1, 1, 3240])
x3 = tf.reshape(x3, [-1, 1, 3240])
x4 = tf.reshape(x4, [-1, 1, 3240])
x5 = tf.reshape(x5, [-1, 1, 3240])
x6 = tf.reshape(x6, [-1, 1, 3240])
x7 = tf.reshape(x7, [-1, 1, 3240])
x8 = tf.reshape(x8, [-1, 1, 3240])
x9 = tf.reshape(x9, [-1, 1, 3240])
x10 = tf.reshape(x10, [-1, 1, 3240])
x11 = tf.reshape(x11, [-1, 1, 3240])
x12 = tf.reshape(x12, [-1, 1, 3240])

output = layers.concatenate([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12], axis=1)#(None, 12,3240)

In [10]:
time_steps_encoder=12
#num_encoder_tokens=3456
num_encoder_tokens=3240
latent_dim=512
time_steps_decoder=24
#time_steps_decoder=3
num_decoder_tokens=1

In [11]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):# 可以在其中进行所有与输入无关的初始化，定义相关的层
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):# 知道输入张量的形状并可以进行其余的初始化
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.
        # 为该层创建一个可训练的权重变量。

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):# 在这里进行前向传播
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state 
                计算单个解码器状态能量的阶跃函数  """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors 整形张量所需的一些参数"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si] 计算 S.Wa"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei 利用ei计算ci的步骤函数 """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            # 我们不使用初始状态，但需要传递一些东西给K.rnn函数
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [19]:
encoder_inputs = output
"""后为特征数，前为批次数量！！！"""
#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_inputs)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
# 我们设定我们的解码器回传整个输出的序列同时也回传内部的states参数
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
# 我们使用`encoder_states`来做为初始值(initial state) <-- 重要
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])


# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

decoder_dense = TimeDistributed(Dense(latent_dim, activation ='tanh'))
decoder_outputs1 = decoder_dense(decoder_concat_input)

decoder_dense2 = TimeDistributed(Dense(num_decoder_tokens))
decoder_outputs2 = decoder_dense2(decoder_outputs1)

# 定义一个模型接收encoder_input_data` & `decoder_input_data`做为输入而输出`decoder_target_data`
model = Model([_input, decoder_inputs], decoder_outputs2)
model.summary()
#plot_model(model, to_file='model_train.png', show_shapes=True, show_layer_names=True)


Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 12, 24, 72)] 0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape_1 (TensorFl [(None, 24, 72)]     0           input_2[0][0]                    
__________________________________________________________________________________________________
functional_1 (Functional)       (None, 3240)         15450       tf_op_layer_Reshape_1[0][0]      
__________________________________________________________________________________________________
tf_op_layer_Reshape_2 (TensorFl [(None, 12, 3240)]   0           functional_1[0][0]               
______________________________________________________________________________________

In [13]:
encoder_inputs = output
# Encoder
#encoder_inputs = Input(shape=(time_steps_encoder,num_encoder_tokens))

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_inputs)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(decoder_inputs,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(latent_dim))
decoder_outputs11 = decoder_dense(decoder_concat_input)

decoder_dense2 =  TimeDistributed(Dense(1))
decoder_outputs = decoder_dense2(decoder_outputs11)

# Define the model 
model = Model([_input, decoder_inputs], decoder_outputs)

model.summary() 
#plot_model(model, to_file='model_train.png', show_shapes=True, show_layer_names=True)

Model: "functional_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 12, 24, 72)] 0                                            
__________________________________________________________________________________________________
tf_op_layer_Reshape_1 (TensorFl [(None, 24, 72)]     0           input_2[0][0]                    
__________________________________________________________________________________________________
functional_1 (Functional)       (None, 3240)         15450       tf_op_layer_Reshape_1[0][0]      
__________________________________________________________________________________________________
tf_op_layer_Reshape_2 (TensorFl [(None, 12, 3240)]   0           functional_1[0][0]               
_______________________________________________________________________________________

In [68]:
def custome_rmse_fn(y_true, y_pred):
    """ custome loss function
    The 24 series is not equally weighted, so log1p weighted is used.
    This is just my initial try, still have further improvement space.

    y_true: batch * 24
    """ 
    #y_pred = y_pred.numpy().reshape(-1,24)
    #y_true = y_true.numpy().reshape(-1,24)
    diff = (y_pred - y_true) ** 2
    predict_sequence_length = tf.shape(y_true)[-1]#获取序列长度
    alpha = [np.log1p(i) for i in range(1, time_steps_decoder+1)]
    #alpha = [np.log(i)*j for i,j in zip(range(1, predict_sequence_length+1), [0.65]*4+[1]*7+[1.2]*7+[1.5]*6)]
    alpha = tf.reshape(tf.convert_to_tensor(alpha, tf.float32), (1, time_steps_decoder))
    #reshape(1,24)
    rmse = tf.sqrt(tf.reduce_mean(diff * alpha))
    return rmse

In [14]:
epochs=100
# Early Stopping
earlystopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights = True)

# Tensorboard callback
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Run training
#opt = keras.optimizers.Adam(lr = 0.0001)
#RMSprop
#当学习停滞时，减少2倍或10倍的学习率常常能获得较好的效果。
x = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,patience=2,verbose=0,mode="auto")

model.compile(metrics=['mae'], optimizer='Adam', loss='mse')

model.fit([trX, trY_decoder_input], trY, batch_size=64, epochs=epochs, validation_split=0.3, verbose=1, callbacks=[earlystopping, x])

Epoch 1/100
64/64 [==============================] - 58s 905ms/step - loss: 0.0874 - mae: 0.1886 - val_loss: 0.0206 - val_mae: 0.1033
Epoch 2/100
64/64 [==============================] - 57s 892ms/step - loss: 0.0187 - mae: 0.1014 - val_loss: 0.0134 - val_mae: 0.0858
Epoch 3/100
64/64 [==============================] - 57s 893ms/step - loss: 0.0133 - mae: 0.0863 - val_loss: 0.0093 - val_mae: 0.0724
Epoch 4/100
64/64 [==============================] - 57s 893ms/step - loss: 0.0102 - mae: 0.0758 - val_loss: 0.0081 - val_mae: 0.0667
Epoch 5/100
64/64 [==============================] - 57s 892ms/step - loss: 0.0088 - mae: 0.0706 - val_loss: 0.0072 - val_mae: 0.0630
Epoch 6/100
64/64 [==============================] - 57s 892ms/step - loss: 0.0078 - mae: 0.0668 - val_loss: 0.0066 - val_mae: 0.0606
Epoch 7/100
64/64 [==============================] - 57s 893ms/step - loss: 0.0074 - mae: 0.0648 - val_loss: 0.0061 - val_mae: 0.0581
Epoch 8/100
64/64 [==============================] - 57s 893ms

In [18]:
# Encode the input sequence to get the feature vector
# 对输入序列进行编码得到特征向量
encoder_model = Model(_input,[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
# 下面的张量将保存上一个时间步的状态
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
#修改
decoder_hidden_state_input = Input(shape=(time_steps_encoder,latent_dim))

# Get the embeddings of the decoder sequence
# 获取解码器序列的嵌入
#dec_emb2= dec_emb_layer(decoder_inputs)
decoder_inputs = Input(shape=(None, num_decoder_tokens))#注意是none (none, 1)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
# 要预测序列中的下一个单词，请将初始状态设置为前一个时间步的状态
decoder_outputs2, state_h2, state_c2 = decoder_lstm(decoder_inputs, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
# 注意力 推理
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
# 用于生成概率分布的密集 softmax 层。 超过目标词汇
decoder_outputs22 = decoder_dense(decoder_inf_concat)
decoder_outputs = decoder_dense2(decoder_outputs22)

# Final decoder model
# 最终解码器模型
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs] + [state_h2, state_c2])


#预测
def decode_predict(input_seq):
    # 将输入编码为状态向量。
    e_out, e_h, e_c = encoder_model.predict(input_seq.reshape(-1, 12, 24, 72))
    #print(e_out.shape)
    # 生成长度为 1 的空目标序列。
    target_seq = np.zeros(( 1,1, num_decoder_tokens))#(1, 1, 1)
    
    # Populate the first word of target sequence with the start word.
    # 用起始词填充目标序列的第一个词。
    target_seq[ 0,0, 0] = 0.

    decoded_sentence = []
    for i in range(24):
        output, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        
        decoded_sentence.append(output[0,0,0])

        # Update the target sequence (of length 1).
        # 更新目标序列（长度为 1）。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq = output
        #print(target_seq)

        # Update internal states
        # 更新内部状态
        states_value = [h, c]

    return decoded_sentence

In [19]:
out = []
for i in range(testX.shape[0]):
    print("test No." + str(i+1))
    out.append(decode_predict(testX[i,:,:]))

test No.1
test No.2
test No.3
test No.4
test No.5
test No.6
test No.7
test No.8
test No.9
test No.10
test No.11
test No.12
test No.13
test No.14
test No.15
test No.16
test No.17
test No.18
test No.19
test No.20
test No.21
test No.22
test No.23
test No.24
test No.25
test No.26
test No.27
test No.28
test No.29
test No.30
test No.31
test No.32
test No.33
test No.34
test No.35
test No.36
test No.37
test No.38
test No.39
test No.40
test No.41
test No.42
test No.43
test No.44
test No.45
test No.46
test No.47
test No.48
test No.49
test No.50
test No.51
test No.52
test No.53
test No.54
test No.55
test No.56
test No.57
test No.58
test No.59
test No.60
test No.61
test No.62
test No.63
test No.64
test No.65
test No.66
test No.67
test No.68
test No.69
test No.70
test No.71
test No.72
test No.73
test No.74
test No.75
test No.76
test No.77
test No.78
test No.79
test No.80
test No.81
test No.82
test No.83
test No.84
test No.85
test No.86
test No.87
test No.88
test No.89
test No.90
test No.91
test No.

In [20]:
cor = np.zeros((24))
for i in range(24):
    cor[i] = np.corrcoef(testY.reshape(384,24)[:,i],np.array(out)[:,i])[0,1]
print(cor)

[ 0.97437555  0.92884431  0.85354765  0.7583023   0.65203463  0.54571141
  0.44398442  0.34470657  0.24869166  0.16245212  0.08885094  0.0250032
 -0.02977233 -0.07126178 -0.10342814 -0.12495972 -0.14000287 -0.15140603
 -0.16489891 -0.18016526 -0.1930057  -0.20139397 -0.20577378 -0.21199596]


In [23]:
out[10]

[-0.85058784,
 -2.3470197,
 -4.9426928,
 -9.195482,
 -15.082142,
 -20.817394,
 -24.295744,
 -25.711657,
 -26.177704,
 -26.319181,
 -26.36103,
 -26.373312,
 -26.376911,
 -26.377964,
 -26.378271,
 -26.378363,
 -26.378387,
 -26.378399,
 -26.378405,
 -26.378405,
 -26.378405,
 -26.378405,
 -26.378405,
 -26.378405]

In [22]:
testY[10]

array([[-1.15875304],
       [-1.15833545],
       [-1.18053198],
       [-1.17541635],
       [-1.00110507],
       [-0.80188906],
       [-0.6772756 ],
       [-0.52563614],
       [-0.43219197],
       [-0.36762857],
       [-0.33536553],
       [-0.28663447],
       [-0.24307424],
       [-0.35159171],
       [-0.53687763],
       [-0.68450022],
       [-0.63430518],
       [-0.45563629],
       [-0.19504076],
       [ 0.00893648],
       [ 0.22462256],
       [ 0.45296106],
       [ 0.66400367],
       [ 0.86413419]])

In [26]:
from sklearn.metrics import mean_squared_error, r2_score
print(f"均方误差(MSE)：{mean_squared_error(out[10], testY[10])}")

均方误差(MSE)：536.300492165276


## 模型预测

In [21]:
save_model_path = 'model_final'
if not os.path.exists(save_model_path):
    os.makedirs(save_model_path)

# Saving encoder as in training
encoder_model = Model(_input, [encoder_outputs, state_h, state_c])

# Saving decoder states and dense layer 
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(time_steps_encoder,latent_dim))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

#attention inference
# 注意力 推理
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs, attn_out_inf])


decoder_outputs11 = decoder_dense(decoder_inf_concat)
decoder_outputs22 = decoder_dense2(decoder_outputs11)
#modle(input, output)
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs22] + decoder_states)

#encoder_model.summary()
#decoder_model.summary()
encoder_model.save(os.path.join(save_model_path, 'encoder_model.h5'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))


In [22]:
# inference encoder model
save_model_path = 'model_final'
inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))
    
# inference decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))#注意是none (none, 1)
decoder_dense1 = Dense(latent_dim,activation ='tanh')
decoder_dense = Dense(num_decoder_tokens)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs, attn_out_inf])

decoder_outputs11 = decoder_dense1(decoder_inf_concat)
decoder_outputs22 = decoder_dense(decoder_outputs11)

inf_decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs22] + decoder_states)

inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

def decode_predict(input_seq):
    # 将输入编码为状态向量，states_value用于decoder
    e_out, e_h, e_c = inf_encoder_model.predict(input_seq.reshape(-1, 12, 24, 72))
    target_seq = np.zeros((1, 1, num_decoder_tokens))#(1, 1, 1)
    target_seq[0, 0, 0] = 0.
    decoded_sentence = []
    
    for i in range(24):
        output, h, c = inf_decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        decoded_sentence.append(output[0,0,0])
      
      # 更新目标序列（长度为1）。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq = output
      
      # Update states
        states_value = [h, c]

    return decoded_sentence

In [40]:
# 将输入编码为状态向量，states_value用于decoder
e_out, e_h, e_c = inf_encoder_model.predict(testX[1,:,:].reshape(-1, 12, 24, 72))
target_seq = np.zeros((1, 1, num_decoder_tokens))#(1, 1, 1)
target_seq[0, 0, 0] = 0.
decoded_sentence = []

In [41]:
for i in range(5):
    output, h, c = inf_decoder_model.predict([target_seq] + [e_out, e_h, e_c])
    print(h)
    decoded_sentence.append(output[0,0,0])
      
    # 更新目标序列（长度为1）。
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq = output
      
    # Update states
    states_value = [h, c]

[[-1.94294360e-02 -4.80716908e-03 -2.43398710e-03  2.57461201e-02
   1.29896495e-02 -3.48316622e-03  2.53515411e-03  9.14517883e-03
   2.73835100e-03  3.65677843e-04  3.84280905e-02  9.89520084e-03
  -9.65179596e-03 -2.10680789e-03  1.35984300e-02 -2.21971665e-02
   4.65500401e-03 -2.84637362e-02 -1.25992149e-02 -3.10753658e-02
   9.89455148e-04 -8.79889540e-03  7.89037999e-03  1.54475163e-06
  -1.70093216e-02  4.07776684e-02  2.14411095e-02  3.68490033e-02
  -3.21448967e-03  3.70949012e-04  7.04183290e-03 -4.41080406e-02
  -1.16891069e-02  5.73234074e-03 -6.44941404e-02  6.92679430e-04
   2.04301011e-02 -1.48590365e-02  3.80915143e-02  4.74250736e-03
   4.26264451e-04 -7.87364785e-03  2.51122732e-02 -1.69392638e-02
  -1.51226399e-02  9.92778223e-03  9.25744884e-03  6.90290052e-03
   4.06714249e-03  1.08147739e-02 -1.79369617e-02  2.14706585e-02
  -2.50345189e-02  1.63761061e-02  9.18420218e-03  2.01003347e-02
   1.64176058e-02  1.16255637e-02  2.78490204e-02 -1.81362834e-02
  -2.34346

In [38]:
testX[i].shape

(72, 24, 12)

In [23]:
out = []
print("test...")
for i in range(testX.shape[0]):
  #print("test No." + str(i+1))
  out.append(decode_predict(testX[i,:,:]))

test...


In [24]:
cor = np.zeros((24))
for i in range(24):
    cor[i] = np.corrcoef(testY.reshape(384,24)[:,i],np.array(out)[:,i])[0,1]
print(cor)

[ 0.97475707  0.92699723  0.85066644  0.75398047  0.64555546  0.53709401
  0.43255995  0.33093848  0.23639528  0.1529586   0.08218984  0.02104576
 -0.02926854 -0.07000558 -0.10031152 -0.11863477 -0.12685242 -0.13249505
 -0.14221413 -0.15461119 -0.16507394 -0.1697786  -0.17384498 -0.18082392]


In [25]:
testY[3]

array([[-0.86499506],
       [-0.88665622],
       [-0.82672739],
       [-0.62346232],
       [-0.70335156],
       [-0.9443807 ],
       [-1.13839138],
       [-1.15875304],
       [-1.15833545],
       [-1.18053198],
       [-1.17541635],
       [-1.00110507],
       [-0.80188906],
       [-0.6772756 ],
       [-0.52563614],
       [-0.43219197],
       [-0.36762857],
       [-0.33536553],
       [-0.28663447],
       [-0.24307424],
       [-0.35159171],
       [-0.53687763],
       [-0.68450022],
       [-0.63430518]])

In [26]:
out[3]

[-0.6586767,
 -1.8683927,
 -4.0427427,
 -7.6841717,
 -12.690783,
 -17.296146,
 -19.81999,
 -20.749151,
 -21.032207,
 -21.112982,
 -21.135582,
 -21.141872,
 -21.143621,
 -21.14411,
 -21.144247,
 -21.144281,
 -21.14429,
 -21.144295,
 -21.144297,
 -21.144297,
 -21.144297,
 -21.144297,
 -21.144297,
 -21.144297]

In [20]:
decode_predict(trX[10,:,:])

[-0.71680033,
 -1.9435332,
 -3.9903831,
 -7.1546187,
 -11.189546,
 -14.781495,
 -16.84002,
 -17.680498,
 -17.969387,
 -18.062328,
 -18.091578,
 -18.100714,
 -18.103563,
 -18.10445,
 -18.104729,
 -18.10481,
 -18.104837,
 -18.104845,
 -18.104847,
 -18.104849,
 -18.10485,
 -18.10485,
 -18.10485,
 -18.10485]

In [21]:
trY[10,:,:]

array([[-0.599],
       [-0.197],
       [ 0.104],
       [ 0.308],
       [ 0.448],
       [ 0.504],
       [ 0.688],
       [ 0.941],
       [ 1.221],
       [ 1.313],
       [ 1.219],
       [ 1.127],
       [ 1.024],
       [ 0.837],
       [ 0.533],
       [ 0.155],
       [-0.3  ],
       [-0.736],
       [-1.032],
       [-1.209],
       [-1.302],
       [-1.454],
       [-1.539],
       [-1.528]])